In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
def mode_within(data_array, percent):
    max_sum_freq = 0
    mode = data_array[0]

    for check_val in set(data_array):
        sum_freq = sum(1 for ele in data_array if abs(ele - check_val) <= abs(percent * check_val / 100))

        if sum_freq > max_sum_freq:
            mode = check_val
            max_sum_freq = sum_freq

    return mode

In [6]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = json.loads(open(data_path, "r").read())['col_name']
data_tensor = torch.reshape(torch.tensor(data),(1,-1, 1))

create_dummy(data_path, dummy_data_path)
dummy_data = json.loads(open(dummy_data_path, "r").read())['col_name']
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1,-1,1))

dummy_theory_output = torch.tensor(mode_within(dummy_data,1))
theory_output = torch.tensor(mode_within(data,1))





In [7]:
scales = [1]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [8]:
print("dummy output: ", dummy_theory_output)
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
    def forward(self,X):
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])
        return (torch.sum(result) == X.size()[1], self.w)

verifier_define_calculation(dummy_data_path, selected_columns,sel_dummy_data_path,verifier_model, verifier_model_path)

dummy output:  tensor(26.2000)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_91066/449633495.py:10: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_91066/449633495.py:10: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])


In [9]:
# prover calculates settings, send to verifier
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])
        return (torch.sum(result) == X.size()[1], self.w)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

Theory_output:  tensor(77.)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_91066/1437681360.py:11: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])


==== Generate & Calibrate Setting ====
scale:  [1]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":1,"param_scale":1,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":300,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,1],"model_input_scales":[1],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_91066/1437681360.py:11: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])


In [10]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0


spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0


Time setup: 1.646347999572754 seconds
Theory output:  tensor(77.)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 77.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[8983262245034475985, 4321099455052339947, 15405541267776549565, 129283864358809647], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [2847721487471738066, 3094933423049675290, 17348522929905272778, 2381998173652826967]]], 'proof': '15744568d7e37b03a18cf8ca3d4a54528e996c0f1ba86a2819c2852307b55aff2dd8d04937c824d0f6e8a8d6e5fd80309cf3f4bc2e01e8a37e7f2a2ba24c6f89071774da94324f2e70283d40ccd1d7e39a1f3fa2f8afaa60191573f62c43e5df1d39bed9e2fafea08db8be6664916ffaf4a74090e2fedabb8ab54c8b7b5053422096a5666d6a57b1afa1ad89802c2318ad2c83668210ce3ac79b743cc36f7f0b18943bcaa58a3da35b69e61267f7e3c9e24aa8f88c1dc33676d54e579454c1cf04579fce0aa8e885329de739c5ee2629a6d2d769e30582f95004fe69596b9bb81bc105d07d26aed722da4096bc8d4dc242c05758addde4de5a3b8612023c50211971e4b3389df3926edd3d7d6673d245e289a0cd70e0def40d8e56cda6ed1d2102d67d589edd3510b783e6ce2d5b9f20c14334c2980c024a8401992661f6976e1449a8a541f60a141f63d787830e9697e1aab251d4b4cbada976cd03dd6c0282

In [11]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)

77.0